In [543]:
import os
from dotenv import load_dotenv
from collections import OrderedDict
import pandas as pd
import numpy as np

import chart_studio
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
import plotly.express as px

import hijri_converter
from hijri_converter import Hijri, Gregorian, convert

In [544]:
# Load environment variables from .env file
load_dotenv()
plotly_chart_studio_username = os.environ['PLOTLY_USERNAME']
plotly_chart_studio_api_key = os.environ['PLOTLY_API_KEY']

# set up connection to chart studio account
chart_studio.tools.set_credentials_file(username=plotly_chart_studio_username, api_key=plotly_chart_studio_api_key)
chart_studio.tools.set_config_file(world_readable=True, sharing='public')

# other global variables
UPLOAD_TO_CHART_STUDIO = True
TOURISM_COMPANY_NAME = "test company"
HIJRI_YEAR = 1444

In [545]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)
        
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg


def save_fig_to_chart_studio(fig, filename, create_copy=True):
    if not UPLOAD_TO_CHART_STUDIO:
        return
    if create_copy:
        fig = go.Figure(fig)
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0), # Set margins to 0
        autosize=True, # makes plot auto resize if embedded as a frame in another application based on the application's defined width/height of the frame
    )  
    filename = f"{TOURISM_COMPANY_NAME} - {HIJRI_YEAR} - {filename}"
    py.plot(fig, filename=filename, auto_open=False)

In [546]:
df = pd.read_excel(f"./monthly_target_dates_df_{HIJRI_YEAR}.xlsx" ,  converters={'Reservation Check-out Date':lambda x:Hijri.fromisoformat(x) ,'Reservation Creation Date':lambda x:Hijri.fromisoformat(x),'Reservation Check-in Date':lambda x:Hijri.fromisoformat(x)})

In [547]:
df.drop(df.index[-1], inplace=True)

In [548]:
df.insert(1, 'Reservation Creation Date (Greg)', df['Reservation Creation Date'].apply(lambda x: hijri_to_greg(x)))

In [549]:
df.drop(columns=['Reservation Creation Date'], inplace=True)

In [550]:
df_line = df.set_index('Reservation Creation Date (Greg)', inplace=False)
#greg_daily_rows_weekly_df.drop(columns=greg_daily_rows_weekly_df.columns[:2], inplace=True)


In [551]:
df_line

,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
Reservation Creation Date (Greg),,,,
2022-07-30,0.0,0.0,0.0,0.0
2022-07-31,0.0,0.0,0.0,0.0
2022-08-01,0.0,0.0,0.0,0.0
2022-08-02,0.0,0.0,0.0,0.0
2022-08-03,0.0,0.0,0.0,0.0
...,...,...,...,...
2023-07-14,NaN,NaN,NaN,645.0
2023-07-15,NaN,NaN,NaN,645.0
2023-07-16,NaN,NaN,NaN,NaN


In [552]:
# note: in px.func(), we set render_mode='svg' as the default is webgl 
# and sometimes it doesn't render when embedding these plots online
fig = px.line(df_line, render_mode='svg')
save_fig_to_chart_studio(fig, 'line - all months')
fig.show()

In [553]:
df= df.melt(id_vars=['Reservation Creation Date (Greg)'], var_name='Target Month', value_name='#Rooms')

In [554]:
df.fillna(0, inplace=True)
df

,Reservation Creation Date (Greg),Target Month,#Rooms
0,2022-07-30,M1 - Ramadhan,0.0
1,2022-07-31,M1 - Ramadhan,0.0
2,2022-08-01,M1 - Ramadhan,0.0
3,2022-08-02,M1 - Ramadhan,0.0
4,2022-08-03,M1 - Ramadhan,0.0
...,...,...,...
1411,2023-07-14,M4 - Dhu al-Hijjah,645.0
1412,2023-07-15,M4 - Dhu al-Hijjah,645.0
1413,2023-07-16,M4 - Dhu al-Hijjah,0.0
1414,2023-07-17,M4 - Dhu al-Hijjah,0.0


Bubble chart 

In [555]:
import plotly.express as px

fig = px.scatter(df, x="Reservation Creation Date (Greg)", y='Target Month',
	         size="#Rooms", color='Target Month',
                 hover_name='Target Month', log_x=False, size_max=80, render_mode='svg')
save_fig_to_chart_studio(fig, 'bubble - all months')
fig.show()


Filled Area Plot

In [566]:
fig = px.area(df, x="Reservation Creation Date (Greg)", y="#Rooms", color="Target Month",
             pattern_shape="Target Month", pattern_shape_sequence=[".", "x", "+","/"], render_mode='svg')

save_fig_to_chart_studio(fig, 'filled area - all months')
fig.show()

TypeError: area() got an unexpected keyword argument 'render_mode'

In [558]:
dummy_start_date = "2019-01-01"
dummy_end_date = "2021-10-03"
# date range from start date to end date and random
# column named value using amount of days as shape
dummy_df = pd.DataFrame({
    "ds": pd.date_range(dummy_start_date, dummy_end_date),
    "value": np.random.randint(low=0, high=30,
size=(pd.to_datetime(dummy_end_date) - pd.to_datetime(dummy_start_date)).days + 1,),
})
dummy_df

,ds,value
0,2019-01-01,22
1,2019-01-02,20
2,2019-01-03,3
3,2019-01-04,8
4,2019-01-05,15
...,...,...
1002,2021-09-29,7
1003,2021-09-30,11
1004,2021-10-01,6
1005,2021-10-02,24


Heatmap calendar chart  

In [559]:
from plotly_calplot import calplot

In [560]:
df

,Reservation Creation Date (Greg),Target Month,#Rooms
0,2022-07-30,M1 - Ramadhan,0.0
1,2022-07-31,M1 - Ramadhan,0.0
2,2022-08-01,M1 - Ramadhan,0.0
3,2022-08-02,M1 - Ramadhan,0.0
4,2022-08-03,M1 - Ramadhan,0.0
...,...,...,...
1411,2023-07-14,M4 - Dhu al-Hijjah,645.0
1412,2023-07-15,M4 - Dhu al-Hijjah,645.0
1413,2023-07-16,M4 - Dhu al-Hijjah,0.0
1414,2023-07-17,M4 - Dhu al-Hijjah,0.0


In [561]:
def get_calplots(df):
    calplots = {}
    for month_name in df['Target Month'].unique():
        df_tmp = df[df['Target Month'] == month_name].drop(columns='Target Month', inplace=False)
        calplots[month_name] = calplot(
            df_tmp, 
            x="Reservation Creation Date (Greg)",
            y="#Rooms",
            colorscale=[
                (0.00, "white"),
                (0.03, "silver"),
                # (0.11, "gray"),
                (0.33, "green"),
                (0.66, "yellow"),
                (1.00, "red")
            ],
            title=month_name,
            gap=2,
            showscale=True,
            years_title=True,
            space_between_plots=0.3,
            month_lines_width=2.5,
        )
        fig.update_layout(
            title_y=0.98, # Adjust the 'y' position (0.0 to 1.0, 0.0 being the bottom, 1.0 being the top)
            xaxis_title="Reservation Creation Date"
        )
    
    return calplots

In [562]:
def get_calplots_separated_by_year(df):
    calplots = {}
    
    for month_name in df['Target Month'].unique():
        
        plots_list = []
        df_tmp = df[
                    (df['Target Month'] == month_name) 
                ].drop(columns='Target Month', inplace=False)
        
        for year in df_tmp['Reservation Creation Date (Greg)'].dt.year.unique():
            df_filtered_tmp = df_tmp[
                    (df_tmp['Reservation Creation Date (Greg)'].dt.year == year)
                ]
            # print(df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month)
            # print(df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month)
            # start_month = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month,
            # end_month = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month,
            # print(start_month)
            # print(end_month)
            diff_months = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month - df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month + 1
            fig = calplot(
                df_filtered_tmp, 
                start_month=df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month,
                end_month=df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month,
                total_height=200,
                x="Reservation Creation Date (Greg)",
                y="#Rooms",
                colorscale=[
                    (0.00, "white"),
                    (0.03, "silver"),
                    # (0.11, "gray"),
                    (0.33, "green"),
                    (0.66, "yellow"),
                    (1.00, "red")
                ],
                title=month_name,
                gap=2,
                showscale=True,
                years_title=True,
                space_between_plots=0.3,
                month_lines_width=2.5,
            )
            fig.update_layout(
                width=120*diff_months,
                title_y=0.98, # Adjust the 'y' position (0.0 to 1.0, 0.0 being the bottom, 1.0 being the top)
                xaxis_title="Reservation Creation Date"
            )
            plots_list.append(fig)

        calplots[month_name] = plots_list
    
    return calplots

In [563]:
# calplots = get_calplots(df)
calplots = get_calplots_separated_by_year(df)

In [564]:
for month_name, figs_list in calplots.items():
    print(month_name)
    if isinstance(figs_list, list):
        for fig in figs_list:
            greg_year = fig.data[0].name
            # month_abbrev = month_name.split(' - ')[0]
            save_fig_to_chart_studio(fig, f'{greg_year} part - heatmap - {month_name}')
            fig.show()
    else:
        fig = figs_list

        greg_years = []
        for go_figure_type in fig.data:
            if isinstance(go_figure_type, plotly.graph_objs._heatmap.Heatmap):
                greg_years.append(go_figure_type.name)
        greg_years = ','.join(greg_years)

        save_fig_to_chart_studio(fig, f'{greg_years} - heatmap - {month_name}')
        fig.show()

M1 - Ramadhan


M2 - Shawwal


M3 - Dhu al-Qi’dah


M4 - Dhu al-Hijjah


Bar chart

In [565]:
fig = px.bar(df, x='Target Month' , y='#Rooms', render_mode='svg')
save_fig_to_chart_studio(fig, 'bar - all months')
fig.show()